# 🧠 Intro to Convolutional Neural Networks (CNNs)
### 📚 A Beginner-Friendly Lesson Using CIFAR-10

Welcome!  
Today we will learn **how computers understand images** using a type of neural network called a **CNN (Convolutional Neural Network)**.

### 🎯 What You Will Learn
- What a CNN is  
- Why convolution is useful for images  
- How to load and visualize the CIFAR-10 dataset  
- How to build a classifier with TensorFlow/Keras  
- How to train, evaluate, and visualize predictions  
- How to modify the model and run your own experiments

# 🖼️ What Is a Convolutional Neural Network?

A **CNN** is a type of neural network designed for images.

### 🤔 How does it work?
Imagine looking at an image. You don't try to understand the whole picture at once —  
your brain examines **small pieces**, like edges, corners, colors, etc.

CNNs do the same thing:

### 🔍 Step 1: Filters scan small patches
A small 3×3 matrix (called a **filter**) slides across the image.

It detects:
- edges  
- corners  
- textures  

### 🧱 Step 2: Build hierarchical features
- Early layers detect simple shapes  
- Later layers detect more complex patterns, like:
  - eyes  
  - wheels  
  - animal shapes  

### 🏁 Step 3: Classifier makes a prediction
The final layer chooses the category with the highest confidence.

---

### 🧠 Why CNNs work well
- They reuse the same filter everywhere → efficient  
- They focus on patterns, not pixel positions  
- They are very good at generalizing  

Now let’s load our dataset!


In [ ]:
# ============================================================
# 📘 Import all required libraries
# ============================================================

# TensorFlow is a deep learning framework we will use to build our CNN.
import tensorflow as tf

# Layers = building blocks (Conv2D, MaxPooling, Dense)
# Models = lets us build a neural network easily
from tensorflow.keras import layers, models

# Used to plot graphs and show images
import matplotlib.pyplot as plt

# For mathematical operations
import numpy as np

# Print TensorFlow version (helpful for debugging)
print("TensorFlow version:", tf.__version__)


# 📦 Loading the CIFAR-10 Dataset

CIFAR-10 is a famous dataset with **60,000 tiny images** (32×32 pixels).  
It contains 10 categories:

- airplane  
- car  
- bird  
- cat  
- deer  
- dog  
- frog  
- horse  
- ship  
- truck  

Let's load it and normalize the pixel values.


In [ ]:
# ============================================================
# 📦 Load the CIFAR-10 dataset from TensorFlow
# This automatically downloads:
#   - x_train: training images
#   - y_train: training labels
#   - x_test: testing images
#   - y_test: testing labels
# ============================================================
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# ============================================================
# 🎨 Normalize the images
# Pixel values in images are between 0 and 255.
# Neural networks learn MUCH better when numbers are small,
# usually between 0 and 1.
#
# Dividing by 255.0 converts every pixel:
#   e.g., 128 → 128/255 ≈ 0.50
#   e.g., 255 → 1.0
#
# This helps the model train faster and more accurately.
# ============================================================
x_train = x_train / 255.0
x_test = x_test / 255.0

# ============================================================
# 🏷️ CIFAR-10 Class Names
# The dataset labels are numbers from 0 to 9.
# Here we map each number to a human-readable name.
#
# Example:
#   0 → airplane
#   1 → car
#   2 → bird
#   ...
# ============================================================
class_names = [
    'airplane',
    'car',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

# ============================================================
# 📐 Show the shapes of our data
# x_train.shape → (50000, 32, 32, 3)
#   - 50000 training images
#   - each image is 32×32 pixels
#   - 3 channels (R, G, B)
#
# y_train.shape → (50000, 1)
#   - each image has 1 label (0–9)
# ============================================================
x_train.shape, y_train.shape


# 👀 Visualizing Sample Images

Before training a model, it helps to **see the data**.

You will notice:
- images are tiny  
- categories vary  
- colors differ  

Let’s plot 16 random images.


In [ ]:
# ============================================================
# 👀 VISUALIZE 16 SAMPLE IMAGES
# Helps students understand what the model is learning.
# ============================================================

plt.figure(figsize=(8,8))  # Create a square 8x8 inch figure

for i in range(16):                # Show 16 images
    plt.subplot(4, 4, i + 1)       # Create a 4×4 grid of images
    plt.imshow(x_train[i])         # Show image number i
    label_index = y_train[i][0]    # Get the numeric label (0–9)
    plt.title(class_names[label_index])  # Show the class name
    plt.axis('off')                # Hide axis numbers (looks cleaner)

plt.show()  # Display all images


# 🏗️ Building a Simple CNN

We will build a CNN with:

1. **Conv2D Layer**  
   - Finds edges & textures

2. **MaxPooling**  
   - Shrinks the image  
   - Keeps important features

3. **Another Conv2D Layer**  
   - Learns more complex shapes  

4. **Flatten + Dense**  
   - Turns image features into a final prediction

This is a very small CNN, perfect for beginners.


In [ ]:
# ============================================================
# 🏗️ BUILDING A SIMPLE CNN
# This CNN has:
#  - 3 convolutional layers
#  - 2 pooling layers
#  - 1 hidden dense layer
#  - 1 output layer
# ============================================================

model = models.Sequential([

    # --------------------------------------------------------
    # 1️⃣ FIRST CONVOLUTIONAL LAYER
    # Conv2D(32 filters, filter size = 3×3)
    # Activation: ReLU (introduces non-linearity)
    # input_shape = size of each image (32, 32, 3)
    # --------------------------------------------------------
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),

    # --------------------------------------------------------
    # 2️⃣ FIRST POOLING LAYER
    # MaxPooling2D(2×2) reduces the image size by half
    # Keeps the strongest features
    # --------------------------------------------------------
    layers.MaxPooling2D((2,2)),

    # --------------------------------------------------------
    # 3️⃣ SECOND CONVOLUTIONAL LAYER
    # Learns more complex features (64 filters now)
    # --------------------------------------------------------
    layers.Conv2D(64, (3,3), activation='relu'),

    # --------------------------------------------------------
    # 4️⃣ SECOND POOLING LAYER
    # Again reduces image size
    # --------------------------------------------------------
    layers.MaxPooling2D((2,2)),

    # --------------------------------------------------------
    # 5️⃣ THIRD CONVOLUTIONAL LAYER
    # Extracts even deeper patterns
    # --------------------------------------------------------
    layers.Conv2D(64, (3,3), activation='relu'),

    # --------------------------------------------------------
    # 6️⃣ FLATTEN LAYER
    # Converts 3D feature maps into a 1D vector
    # so it can go into Dense layers
    # --------------------------------------------------------
    layers.Flatten(),

    # --------------------------------------------------------
    # 7️⃣ FULLY CONNECTED LAYER (Dense)
    # 64 neurons → learns final combinations of features
    # --------------------------------------------------------
    layers.Dense(64, activation='relu'),

    # --------------------------------------------------------
    # 8️⃣ OUTPUT LAYER
    # 10 neurons = 10 classes
    # softmax → probabilities add up to 1
    # --------------------------------------------------------
    layers.Dense(10, activation='softmax')
])

# Show a summary of the model architecture
model.summary()


# 🚀 Training the CNN

We will train for **10 epochs**.

- `optimizer='adam'` → helps the model learn  
- `loss='sparse_categorical_crossentropy'` → for multi-class labels  
- `metrics=['accuracy']` → evaluate performance  

Let's start training!


In [ ]:
# ============================================================
# ⚙️ COMPILE THE MODEL
# optimizer='adam' → adjusts learning rate automatically
# loss='sparse_categorical_crossentropy' → good for integer labels
# metrics=['accuracy'] → measure how well model performs
# ============================================================

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ============================================================
# 🚀 TRAIN THE MODEL
# history object stores accuracy and loss values
# epochs=10 → the model will see the whole dataset 10 times
# validation_data → test accuracy during training
# ============================================================

history = model.fit(
    x_train,            # Input images
    y_train,            # Labels for training images
    epochs=10,          # Number of passes over the entire dataset
    validation_data=(x_test, y_test)   # Check performance on test data
)


# 📊 Visualizing Training Progress

These plots show:
- How well the model learns  
- Whether it overfits  
- Validation vs training accuracy  

Let’s plot the curves.


In [ ]:
# ============================================================
# 📊 PLOT TRAINING & VALIDATION PERFORMANCE
# Helps visualize:
#  - How well the model is learning
#  - If it is overfitting or underfitting
# ============================================================

plt.figure(figsize=(12,5))

# ------------------------------------------------------------
# Accuracy Plot
# ------------------------------------------------------------
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# ------------------------------------------------------------
# Loss Plot
# ------------------------------------------------------------
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()


# 🔍 Making Predictions

Now the fun part!

We let the model guess what an image is.  
We will:

1. Pick an image  
2. Show the actual label  
3. Show the prediction  


In [ ]:
# ============================================================
# 🔍 USE MODEL TO MAKE PREDICTIONS
# model.predict() returns probabilities for each class
# ============================================================

predictions = model.predict(x_test)

# Function to show an image + prediction
def show_prediction(i):

    plt.imshow(x_test[i])   # Show the image

    # Predicted label (highest probability)
    predicted_label = np.argmax(predictions[i])

    # True label
    true_label = y_test[i][0]

    plt.title(
        f"Prediction: {class_names[predicted_label]}\n"
        f"Actual: {class_names[true_label]}"
    )
    plt.axis('off')

# Show prediction for image 0
show_prediction(0)
